In [1]:
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [5]:
import requests
import os
import csv
import json
import openai
from openai import OpenAI

client = OpenAI()

# 1) Upstage에서 발급받은 API 키를 입력해 주세요.
api_key = os.getenv("UPSTAGE_API_KEY")

# 2) Upstage OCR API, Chat Completion API 엔드포인트 설정
ocr_url = "https://api.upstage.ai/v1/document-ai/ocr"

headers_ocr = {"Authorization": f"Bearer {api_key}"}

# 1) prompt/ocr04.yaml 파일을 읽어오기
with open("prompt/ocr_o3.yaml", "r", encoding="utf-8") as f:
    base_prompt = f.read()

# 3) OCR할 이미지가 들어있는 폴더 경로
image_folder = "image"
image_files = [f for f in os.listdir(image_folder) if f.endswith((".jpg", ".png"))]

# 4) For 문을 돌면서 각 이미지에 대해 OCR 수행 후, 그 결과를 Chat Completion 모델에 전달
for filename in image_files:
    # 이미지 열기
    with open(os.path.join(image_folder, filename), "rb") as file:
        files = {"document": file}
        # OCR 요청
        response_ocr = requests.post(ocr_url, headers=headers_ocr, files=files)
        response_ocr.raise_for_status()  # 4xx, 5xx 에러 시 예외 발생
        ocr_data = response_ocr.json()

    # OCR 결과에서 텍스트 추출
    pages = ocr_data.get("pages", [])
    if not pages:
        print(f"'{filename}'에서 OCR 결과 'pages'가 비어 있습니다.")
        continue

    # 여러 페이지가 있을 수 있으니 모두 이어붙이거나 필요한 부분만 선택
    ocr_texts = []
    for page in pages:
        text = page.get("text", "")
        ocr_texts.append(text)
    joined_text = "\n".join(ocr_texts)

    # 4-2) ocr04.yaml + OCR 텍스트를 결합하여 prompt 작성
    prompt_text = (
        f"{base_prompt}\n\n"
        f"context: {joined_text}\n\n"
        "context를 csv 형식으로 변환해줘"
    )

    # 4-3) openai.Chat.create(...) 사용 (openai>=1.0.0)
    # o3-mini 모델에 메시지를 전달할 때 temperature 파라미터는 생략
    response = client.chat.completions.create(
        model="o3-mini",
        reasoning_effort="medium",
        messages=[
            {"role": "system", "content": prompt_text},
            {"role": "user", "content": "csv로 변환해주세요"},
        ],
    )

    # 응답 파싱
    if not response.choices:
        print(f"'{filename}' 파일 모델 응답이 비어 있습니다.")
        continue

    result_text = response.choices[0].message.content

    # 결과를 CSV 파일로 저장
    csv_filename = f"{os.path.splitext(filename)[0]}.csv"
    csv_lines = result_text.split("\n")

    # 만약 특정 구분선("발주 주체" 이후만 저장)을 사용하고 싶다면 아래 필터를 추가
    filtered_data = []
    record = False
    for line in csv_lines:
        if "발주 주체" in line:
            record = True
        if record:
            filtered_data.append(line)

    with open(csv_filename, mode="w", newline="", encoding="utf-8-sig") as file:
        writer = csv.writer(file)
        for row in filtered_data:
            # 여러 열이 있을 것을 가정하고 콤마로 split
            writer.writerow(row.split(","))

    print(f"CSV 파일 '{csv_filename}'이(가) 성공적으로 저장되었습니다.")

CSV 파일 'Lotte mart 3월 2차 행사리스트(11).csv'이(가) 성공적으로 저장되었습니다.
CSV 파일 'Lotte mart 3월 2차 행사리스트(12).csv'이(가) 성공적으로 저장되었습니다.
CSV 파일 'Lotte mart 3월 2차 행사리스트(6).csv'이(가) 성공적으로 저장되었습니다.
CSV 파일 'Lotte mart 3월 2차 행사리스트(7).csv'이(가) 성공적으로 저장되었습니다.
